# Zero knowledge proof examples
### We need to install python-libsnark

In [1]:
!pip install python-libsnark

     |████████████████████████████████| 2.5MB 4.4MB/s 


# Example 1. Mutual Fund example
The active mutual fund manager wants to prove that the fund contains only using certain stocks, eg. { GOOG, AAPL } and nothing else. But the manager wants to hide the individual stock positions.<br/><br/>
 
 <b>Constraint breakdown:</b><br/><i>
 partial_sum1 = goog_position * goog_price<br/>
 partial_sum2 = aapl_position * aapl_price<br/>
 price = partial_sum1 + partial_sum2<br/></i>

In [2]:
import libsnark.alt_bn128 as libsnark
def mutual_fund_constraint(pb):
  # position variables
  # they are private for the fund manager
  goog_position = libsnark.PbVariable()
  goog_position.allocate(pb)

  aapl_position = libsnark.PbVariable()
  aapl_position.allocate(pb)

  # partials
  partial_sum1 = libsnark.PbVariable()
  partial_sum1.allocate(pb)
  partial_sum2 = libsnark.PbVariable()
  partial_sum2.allocate(pb)

  # daily close price variables
  # they are public and they will used by verifiers
  goog_price = libsnark.PbVariable()
  goog_price.allocate(pb)
  pb.setpublic(goog_price)

  aapl_price = libsnark.PbVariable()
  aapl_price.allocate(pb)
  pb.setpublic(aapl_price)

  # fund closing price
  fund_price = libsnark.PbVariable()
  fund_price.allocate(pb)
  pb.setpublic(fund_price)

  # partial1 = goog_price * goog_position
  pb.add_r1cs_constraint(libsnark.R1csConstraint(libsnark.LinearCombination(goog_price),
                                                      libsnark.LinearCombination(goog_position),
                                                      libsnark.LinearCombination(partial_sum1)))
  # partial2 = aapl_price * aapl_position
  pb.add_r1cs_constraint(libsnark.R1csConstraint(libsnark.LinearCombination(aapl_price),
                                                      libsnark.LinearCombination(aapl_position),
                                                      libsnark.LinearCombination(partial_sum2)))
  # fund  price = (partial_sum1 + partial_sum2) * 1
  pb.add_r1cs_constraint(libsnark.R1csConstraint(libsnark.LinearCombination(partial_sum1) + libsnark.LinearCombination(partial_sum2),
                                                      libsnark.LinearCombination(1),
                                                      libsnark.LinearCombination(fund_price)))

  return fund_price, goog_price, aapl_price, goog_position, aapl_position, partial_sum1, partial_sum2

## Generate prooving and verification keys

In [3]:
  # creating a new ZNP board
  pb=libsnark.ProtoboardPub()

  mutual_fund_constraint(pb)
  cs=pb.get_constraint_system_pubs()
  keypair=libsnark.zk_generator(cs)
  libsnark.zk_write_keys(keypair, "fund.vk", "fund.ek")

### Mutual Fund proof 

In [4]:
# creating a new ZNP board
pb=libsnark.ProtoboardPub()
fund_price, goog_price, aapl_price, goog_position, aapl_position, partial_sum1, partial_sum2 = mutual_fund_constraint(pb)

# now the fund managers sets all the variables
# private variables
pb.setval(goog_position, 10 )
pb.setval(aapl_position, 20 )
pb.setval(partial_sum1, 1465 * 10 )
pb.setval(partial_sum2, 437 * 20 )
# public variables
pb.setval(goog_price, 1465 )
pb.setval(aapl_price, 437 )
# 1465 * 10 + 437 * 20 = 23390
pb.setval(fund_price, 23390)

cs=pb.get_constraint_system_pubs()
pubvals=pb.primary_input_pubs();
privvals=pb.auxiliary_input_pubs();

# read the key
keypair=libsnark.zk_read_key("fund.ek", cs)
if not keypair:
    raise Exception("*** No prooving key or computation changed")    

print("*** Private inputs: " + " ".join([str(privvals.at(i)) for i in range(privvals.size())]))
print("*** Public inputs: " + " ".join([str(pubvals.at(i)) for i in range(pubvals.size())]))
proof=libsnark.zk_prover(keypair.pk, pubvals, privvals);
libsnark.zk_write_proof(proof,pubvals,'proof')
!ls -lart

*** Private inputs: 10 20 14650 8740
*** Public inputs: 1465 437 23390
total 36
drwxr-xr-x 1 root root  4096 Jul 30 16:30 sample_data
drwxr-xr-x 1 root root  4096 Aug  3 16:17 .config
drwxr-xr-x 1 root root  4096 Aug  6 04:29 ..
-rw-r--r-- 1 root root  2486 Aug  6 04:30 fund.vk
-rw-r--r-- 1 root root 10584 Aug  6 04:30 fund.ek
-rw-r--r-- 1 root root  1414 Aug  6 04:30 proof
drwxr-xr-x 1 root root  4096 Aug  6 04:30 .


## Proof Verification


In [5]:
# creating a new ZNP board
pb=libsnark.ProtoboardPub()
fund_price, goog_price, aapl_price, _, _, _, _ = mutual_fund_constraint(pb)

cs=pb.get_constraint_system_pubs()
keypair=libsnark.zk_read_key("fund.ek", cs)
if not keypair:
    raise Exception("*** No verification key or computation changed")  

# public variables
pb.setval(goog_price, 1465 )
pb.setval(aapl_price, 437 )
pb.setval(fund_price, 23390)

pubvals=pb.primary_input_pubs();
verified=libsnark.zk_verifier_strong_IC(keypair.vk, pubvals, proof);
    
print("*** Public inputs: " + " ".join([str(pubvals.at(i)) for i in range(pubvals.size())]))
print("*** Verification status:", verified)

*** Public inputs: 1465 437 23390
*** Verification status: True


#  Example 2: Sufficient Funds Verification <br/>
The Banks ABC's client has at least N dollars and her/his account 
<br> The Bank will compare the client account balance (private data) with the requested threshold (public data) and generates the public proof.
<br/>
This is an example of the comparision constraint

In [6]:
import libsnark.alt_bn128 as libsnark

def signed_2_unsigned(arg, word_size = 32):
  return arg if arg >= 0 else arg + 2**word_size

def bitfield(n):
    return [int(digit) for digit in bin(n)[2:][::-1]] # [2:] to chop off the "0b" part, and reverse

def bin_constraint(pb, word_size):
    # decimal value
    packed = libsnark.PbVariable()
    packed.allocate(pb)
    # array of bin representation of the decimal value
    unpacked_array = []
    for n in range(word_size):
      v = libsnark.PbVariable()
      v.allocate(pb)
      unpacked_array.append(v)

    # bin to decimal calculation
    packed_combination = libsnark.LinearCombination(0);
    for n in range(word_size):
      packed_combination += libsnark.LinearCombination(unpacked_array[n]) * 2**n

    # add constrain packed == packed_combination
    pb.add_r1cs_constraint(libsnark.R1csConstraint( packed_combination,
                                                    libsnark.LinearCombination(1),
                                                    libsnark.LinearCombination(packed)))
    return unpacked_array, packed

# Compare
# Given two positive numbers A, B, calculate R = A-B
# if A>=B, then R >= 0, so the sign bit will 0 meaning unpacked_array[0] == 0
# otherwise the unpacked_array[0] == 1
#
# The inputs will be: A, B, unsigned R, binary array of R: b(R)
# We have to prove that
# 1. decimal unsigned R equals binary b(R), bin_constraint
# 2. signed R  == A - B, which is linear constraint
# 3. b(R)[0] == 1 ( A >= B )
def less_constraint(pb, word_size = 32):

    # 1. decimal R equals binary b(R), bin_constraint
    b_r, r_unsigned = bin_constraint(pb, word_size)

    # 2. R == A - B
    a = libsnark.PbVariable()
    a.allocate(pb)
    pb.setpublic(a)

    b = libsnark.PbVariable()
    b.allocate(pb)
    

    pb.add_r1cs_constraint(libsnark.R1csConstraint( libsnark.LinearCombination(a) - libsnark.LinearCombination(b),
                                                    libsnark.LinearCombination(1),
                                                    # convert it back to signed
                                                    libsnark.LinearCombination(r_unsigned) - libsnark.LinearCombination(2**word_size) ))


    # 3. b(R)[word_size - 1] == 1
    pb.add_r1cs_constraint(libsnark.R1csConstraint( libsnark.LinearCombination(b_r[word_size - 1]),
                                                    libsnark.LinearCombination(1),
                                                    libsnark.LinearCombination(1)))
    
    return a, b, r_unsigned, b_r


## Testing the comparison constraint
#### Client's balance: 1000 USD
#### Required balance: 900 USD

In [11]:
# creating the board
pb=libsnark.ProtoboardPub()

# define compare constraints and get the created variables back
a, b, r_unsigned, b_r = less_constraint(pb)

# generate a pair of proving and verification keys for the compare constraints
keypair=libsnark.zk_generator(cs)


val_clients_balance = 1000
val_required_threshold = 900
val_unsigned_r = signed_2_unsigned(val_required_threshold - val_clients_balance )

bit_array = bitfield(val_unsigned_r)
print(bit_array)
for n, bit in zip(range(len(bit_array)), bit_array):
  pb.setval(b_r[n], bit)

pb.setval(r_unsigned, val_unsigned_r )
pb.setval(a, val_required_threshold )
pb.setval(b, val_clients_balance )



cs=pb.get_constraint_system_pubs()
pubvals=pb.primary_input_pubs();
privvals=pb.auxiliary_input_pubs();

proof=libsnark.zk_prover(keypair.pk, pubvals, privvals);

verified=libsnark.zk_verifier_strong_IC(keypair.vk, pubvals, proof);
    
print("*** Public inputs: " + " ".join([str(pubvals.at(i)) for i in range(pubvals.size())]))
print("*** Verification status:", verified)

[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
*** Public inputs: 900
*** Verification status: True
